# The Numeric Widgets

#### `Int` Widgets

+ IntText
+ BoundedIntText
+ IntProgress
+ IntSlider
+ IntRangeSlider
+ Play

#### `Float` Widgets

+ FloatText
+ BoundedFloatText
+ FloatProgress
+ FloatSlider
+ FloatRangeSlider

----------------------

First of all, we have to import the Widgets library. Sometimes we need some Haskell language extensions. We need `OverloadedStrings` because some widget methods receive a `Text` as an argument, instead of a `[Char]`.

In [1]:
{-# LANGUAGE OverloadedStrings #-}
import IHaskell.Display.Widgets

#### `IntText` and `FloatText`
We are going to see how the `Text` family works. They create a "Stepper", which lets us click to increment a certain amount. We can also input the number as if it were text.

In [2]:
intt <- mkIntText
floatt <- mkFloatText
intt
floatt

For any numeric widget, we can use `getField w IntValue` and `getField w FloatValue` to obtain the value stored in the widget. Try changing the numbers in the widgets and then executing the cell bellow!

In [3]:
getField @IntValue intt
getField @FloatValue floatt

0

0.0

We can also use these results for any computation. For example, let us add them together:

In [4]:
x <- getField @IntValue intt
y <- getField @FloatValue floatt

(fromIntegral x) + y

0.0

But incrementing and decrementing one by one is a bit boring... We can change the number incremented each step if we set the `StepInt` and `StepFloat` fields. They are of type `Maybe Integer`, so we can put `Nothing` and let the frontend decide.

In [5]:
setField @StepInt intt $ Just 5
setField @StepFloat floatt $ Just 0.5

Try clicking on the buttons of the text field now.

By the way, does this mean that we can change other fields with `setField`? Yes! For example, let us change the `IntValue`/`FloatValue`

In [6]:
setField @IntValue intt 42
setField @FloatValue floatt 3.14

One special field is the `ChangeHandler`. This field is an IO function that is executed everytime the value is changed. We are going to "sync" the two widgets using two `ChangeHandler` functions. Each time the value changes in one widget, it is changed on the other widget too. In this example, the int widget contains the doubled value of the float widget.

In [7]:
setField @ChangeHandler intt (getField @IntValue intt >>= setField @FloatValue floatt . (/2) . fromIntegral)
setField @ChangeHandler floatt (getField @FloatValue floatt >>= setField @IntValue intt . (*2) . round)

-- Let's display the widgets again so we don't have to scroll up and down
intt
floatt

#### `BoundedIntText` and `BoundedFloatText`

So, what's the difference between the Bounded family and the other two? `Bounded` widgets have the `Max` and `Min` attributes, which let you set an upper and lower bound. Let's try it

In [8]:
-- We create the two widgets
bit <- mkBoundedIntText
bft <- mkBoundedFloatText

setField @MaxInt bit 5
setField @MinInt bit (-5)

setField @MaxFloat bft 20
setField @MinFloat bft 10
setField @StepFloat bft $ Just 0.5
setField @FloatValue bft 15

bit
bft

If you try clicking on the buttons or editing the text, you can't but anything that is not between the `Min` and `Max`! That's pretty neat, but, if you want to do something with bounds, maybe it's better to use an slider

#### `IntSlider`, `FloatSlider`, `FloatLogSlider`, `IntRangeSlider` and `FloatRangeSlider`

All these widgets are sliders (duh!). `IntSlider`, `FloatSlider` and `FloatLogSlider` represent a single value, whereas `IntRangeSlider` and `FloatRangeSlider` represent a pair (range) of values. `FloatLogSlider` uses a logarithmic scale, which means that every step will multiply (instead of increment) the value!

In [9]:
ins <- mkIntSlider
irs <- mkIntRangeSlider
fns <- mkFloatSlider
fls <- mkFloatLogSlider
frs <- mkFloatRangeSlider
setField @StepFloat fns $ Just 0.25
-- We can set the base of the logslider
setField @BaseFloat fls 2
setField @StepFloat fls $ Just 1

In [10]:
ins
irs
fns
fls
frs

They work the same as the previous widgets plus, they are a lot more confortable if you need to have some kind of bounds. The only difference is that with the pair ones, we have to get the `IntPairValue` and `FloatPairValue` fields

In [11]:
getField @IntPairValue irs
getField @FloatPairValue frs

(25,75)

(0.0,1.0)

In [12]:
-- We can also set the field!
setField @IntPairValue irs (32,42)

Let's create a small program using widgets that gives us the greatest common divisor of two numbers! It displays the result on a `IntText` widget

In [13]:
gcd a 0 = a
gcd a b = gcd b $ a `mod` b

setField @ChangeHandler irs (getField @IntPairValue irs >>= setField @IntValue intt . uncurry gcd)

irs
intt

**Bonus: ContinuousUpdate**

Did you realize that the value in the `IntText` widget changed while you were moving the slider? If you have a very time-consuming computation, maybe that is a very bad idea. For this, you can turn off the `ContinuousUpdate`. If you turn it off, the value will be computed only when you stop sliding the slider. But this is more easily understood if we test it!

In [14]:
setField @ContinuousUpdate irs False

**Bonus: Disable**

There are some widgets that can be disabled, so they appear with a different style and are not modifiable.

In [15]:
setField @Disabled irs True

**Bonus: Readout**

You see those little numbers at the side of the widget? They are called the `ReadOut`, you can disable it or change the format (using a format similar to printf).

In [16]:
-- First we're gonna enable the widget again
setField @Disabled irs False
-- We add 4 trailing zeroes
setField @ReadOutFormat irs "04d"

-- We can disable the readout with

-- setField irs ReadOut False

#### `IntProgress` and `FloatProgress`

Finally, we have these two widgets, that we can use as progress bars.

In [17]:
fnp <- mkFloatProgress
fnp

In [18]:
setField @FloatValue fnp 42.5
getField @FloatValue fnp

42.5

We can also display them Vertically.

*(Did you know you can also display the Sliders and other widgets vertically?)*

In [19]:
setField @Orientation fnp VerticalOrientation

Now we are going to create a REAL progress bar! For example, let's create a thread that does some incredible complex calculations (sleeping) and then reports the progress to the widget

In [20]:
import Control.Concurrent
import System.IO.Unsafe

inp <- mkIntProgress
inp

In [52]:
f :: Integer -> IO ()
f x = do
    threadDelay $ 10^5
    setField @IntValue inp x
   
thid <- forkIO $ mapM_ f [0..100]

### The `Play` widget

This widgets display a play/repeat buttons that step through values automatically (and optionally loop)

In [ ]:
play <- mkPlay
setField @ChangeHandler play (getField @IntValue play >>= setField @IntValue inp)
play